In [236]:
import csv
import tf2onnx
import onnx

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [237]:
dataset = 'model/handClassifier/keypoint.csv'
model_save_path = 'model/handClassifier/keypoint_classifier.keras'
tflite_save_path = 'model/handClassifier/keypoint_classifier.tflite'

# Set number of classes

In [238]:
NUM_CLASSES = 6

# Dataset reading

In [239]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [240]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [241]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=42)

# Model building

In [242]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [243]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_28 (Dropout)            │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 20)             │           860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 6)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,136 (4.44 KB)

 Trainable params: 1,136 (4.44 KB)

 Non-trainable params: 0 (0.00 B)

In [244]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [245]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [246]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
 1/31 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.0625 - loss: 1.8883
Epoch 1: saving model to model/handClassifier/keypoint_classifier.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.1555 - loss: 1.8101 - val_accuracy: 0.3746 - val_loss: 1.6128
Epoch 2/1000
 1/31 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3516 - loss: 1.6297
Epoch 2: saving model to model/handClassifier/keypoint_classifier.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3061 - loss: 1.6105 - val_accuracy: 0.4187 - val_loss: 1.4564
Epoch 3/1000
 1/31 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3828 - loss: 1.4234
Epoch 3: saving model to model/handClassifier/keypoint_classifier.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3386 - loss: 1.4958 - val_accuracy: 0.4559 - val_loss: 1.3655
Epoch 4/1000
 1/31 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3281 - loss: 1.4505
Epoch 4: saving model to model/handClassifier/keypoint_classifier.keras
31/31 ━━━━━━━━━━━━━━

In [247]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - accuracy: 0.9528 - loss: 0.3172


In [248]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [249]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
[8.1036657e-02 4.8517120e-01 4.2618340e-01 3.5531641e-05 3.0804316e-03
 4.4929055e-03]
1


# Convert to model for Tensorflow-Lite

In [250]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [251]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: C:\Users\narte\AppData\Local\Temp\tmpehkx5s4o\assets


INFO:tensorflow:Assets written to: C:\Users\narte\AppData\Local\Temp\tmpehkx5s4o\assets


Saved artifact at 'C:\Users\narte\AppData\Local\Temp\tmpehkx5s4o'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 42), dtype=tf.float32, name='input_layer_14')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  2230253430800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2230253428112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2230341474192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2230341472848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2230341481104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2230341471120: TensorSpec(shape=(), dtype=tf.resource, name=None)


6660

# Inference test

In [252]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [253]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [254]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [255]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

CPU times: total: 0 ns
Wall time: 0 ns


In [256]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[8.1036672e-02 4.8517114e-01 4.2618334e-01 3.5531673e-05 3.0804311e-03
 4.4929050e-03]
1
